This notebook is focused on users. You can:
- [Fetch all languages of a specific user](#languages_of_user)
- [Get the list of users speaking a specific language](#users_of_language) 
- [Get the list of native speakers of a specific language](#natives_of_language)
- [Get a list of native speakers of one language speaking another](#natives_speaking)

Before experimenting with any of the options described above, it is necessary to set and execute the cells under the [Languages of users](#languages_of_users) section.

If you're new to Jupyter, please click on `Cell > Run All` from the top menu to see what the notebook does. You should see that cells that are running have an `In[*]` that will become `In[n]` when their execution is finished (`n` is a number). To run a specific cell, click in it and press `Shift + Enter` or click the `Run` button of the top menu. 

In any case, to be able to use the notebook correctly, please run the two following cells first.

In [ ]:
import pandas as pd
import csv
import tarfile

In [ ]:
pd.set_option('display.max_colwidth', -1) # To display full content of the column
# pd.set_option('display.max_rows', None) # To display ALL rows of the dataframe (otherwise you can decide the max number)

<a id='languages_of_users'></a>
# Languages of users

Run the two following cells (you don't have to modify them).

In [ ]:
def read_user_languages():
    data = pd.read_csv('./user_languages.csv', 
            sep='\t', 
            header=None, 
            names=['Language', 'Level', 'Username', 'Details'],
            quoting=csv.QUOTE_NONE)
    # Remove unknown users
    data = data[data['Username'] != r'\N']
    data = data.dropna(subset=['Username'])
    return data.fillna('')

user_infos = read_user_languages()
print(f"{len(user_infos):,} users found.")

The cell below displays 10 random rows, just to give you an overview of the structure of the data.

In [ ]:
user_infos.sample(10)

<a id='languages_of_user'></a>
### Languages of a specific user

Run the two following cells (you don't have to modify them).

In [ ]:
def languages_of_user(username, user_frame):
    return user_frame[user_frame['Username'] == username].sort_values(by='Level', ascending=False)

Replace `username` by the username you want to check, and run the following cell. The results are displayed by descending `Level` order.

In [ ]:
username = 'nimfeo'  # <-- Modify this value
languages_of_user(username, user_infos)

<a id='users_of_language'></a>
### Users of a specific language

Run the two following cells. You don't have to modify them unless you want to sort the final results by an attribute other than `Username`.

In [ ]:
def users_of_language(iso, user_frame):
    frame = user_frame[user_frame['Language'] == iso].sort_values(by='Username')
    print(f"{len(frame):,} users found.")
    return frame

Specify your target language as a 3-letter ISO code (`cmn`, `fra`, `jpn`, `eng`, etc.) and run the next cell to obtain the list of all its speakers.

In [ ]:
language = 'fra'  # <-- Modify this value
users_of_language(language, user_infos)

<a id='natives_of_language'></a>
### Natives of a specific language

Run the two following cells (you don't have to modify them).

In [ ]:
def natives_of_language(iso, user_frame):
    frame = user_frame[user_frame['Language'] == iso].sort_values(by='Username')
    frame = frame[frame['Level'] == '5']
    print(f"{len(frame):,} native users found.")
    return frame

Specify your target language as a 3-letter ISO code (`cmn`, `fra`, `jpn`, `eng`, etc.) and run the following cells to obtain the list of all its native speakers.

In [ ]:
language = 'fra'  # <-- Modify this value
natives_of_language(language, user_infos)

<a id='natives_speaking'></a>
### Natives of X speaking Y

Run the following cell (you don't have to modify it).

In [ ]:
def natives_speaking_other(main_language, other_language, user_frame):
    native_frame = user_frame[user_frame['Language'] == main_language].sort_values(by='Username')
    native_users = native_frame[native_frame['Level'] == '5'].Username.tolist()
    second_frame = user_frame[user_frame['Language'] == other_language]
    second_users = second_frame.Username.tolist()
    target_users = list(set(native_users).intersection(second_users))
    result = user_frame[user_frame['Username'].isin(target_users) & user_frame['Language'].isin([main_language, other_language])].sort_values(by='Username')
    print(f'{len(result) // 2:,} users found.')
    return result

The following cell will fetch users who are natives in `main_language` but also speak `other_language`.  
Specify your target language as a 3-letter ISO code (`cmn`, `fra`, `jpn`, `eng`, etc.) and run the cell.

In [ ]:
main_language = 'fra'  # <-- Modify this value
other_language = 'eng'  # <-- Modify this value
natives_speaking_other(main_language, other_language, user_infos)

You can get the list of usernames from any frame you built by appending `.Username.tolist()`. A list is easier to export and work with. Try it for the natives of X speaking Y you built above:  

In [ ]:
natives_speaking_other(main_language, other_language, user_infos).Username.tolist()